In [1]:
from agents import WebSearchTool, trace, Agent, Runner, gen_trace_id, function_tool
from agents.model_settings import ModelSettings
from pydantic import BaseModel
from dotenv import load_dotenv
import asyncio
import sendgrid
import os
from sendgrid.helpers.mail import Mail, Email, To, Content
from typing import Dict
from IPython.display import Markdown, display

In [2]:
load_dotenv(override=True)

True

In [3]:
INSTRUCTIONS = "You are a research assistant. Given a search term, you search the web for that term and \
produce a concise summary of the results. The summary must 2-3 paragraphs and less than 300 \
words. Capture the main points. Write succintly, no need to have complete sentences or good \
grammar. This will be consumed by someone synthesizing a report, so it's vital you capture the \
essence and ignore any fluff. Do not include any additional commentary other than the summary itself."

In [4]:
model = "gpt-4o-mini"
web_model = "gpt-5-nano"

In [5]:
search_agent = Agent(
    name="Search Agent",
    model=web_model,
    instructions=INSTRUCTIONS,
    tools=[WebSearchTool(search_context_size="low")],
    model_settings=ModelSettings(tool_choice="required")
)

In [10]:
message = "Latest AI frameworks in 2025"

with trace("Search"):
    result = await Runner.run(search_agent, message)

    display(Markdown(result.final_output))

Agent-centric AI frameworks solidify 2025. Pocketflow emerges as a Python framework for Human-AI co-design, with flow-based orchestration, modular Nodes, Flow-as-Node, and explicit control to manage complex agent workflows. LightAgent, a lightweight open-source framework, adds memory, tools, and Tree-of-Thought, with a 0.4.0 release in June 2025 and MCP protocol integration for cross-model tool use. ([arxiv.org](https://arxiv.org/abs/2504.03771?utm_source=openai))

Beyond orchestration, RL-driven and testing frameworks proliferate. Agent Lightning presents a flexible approach to train any AI agent with reinforcement learning, decoupling agent execution from training to work with existing stacks (e.g., LangChain, AutoGen); published Aug 2025. DoomArena provides a plug-in framework to stress-test AI agents against evolving security threats, enabling configurable threat models. PiFlow proposes a principled, information-theoretic multi-agent framework for automated scientific discovery. ([arxiv.org](https://arxiv.org/abs/2508.03680?utm_source=openai))

Deployment/inference frameworks advance as well. Intel’s OpenVINO 2025.3 (Sept 3, 2025) expands GenAI coverage, adds model support, and boosts NPU integration for efficient edge/cloud/local deployments. This release tightens model conversion, hardware acceleration, and pipeline capabilities for agentic workflows. ([docs.openvino.ai](https://docs.openvino.ai/2025/about-openvino/release-notes-openvino.html?utm_source=openai))

In [17]:
HOW_MANY_SEARCHES = 1

INSTRUCTIONS = f"You are a helpful research assistant. Given a query, come up with a set of web searches \
to perform to best answer the query. Output {HOW_MANY_SEARCHES} terms to query for."

class WebSearchItem(BaseModel):
    reason:str
    "Your reasoning for why this search term is important to the query"

    query:str
    "The search term to use for the web search"

class WebSearchPlan(BaseModel):
    searches: list[WebSearchItem]
    """A list of web searches to perform to best answer the query"""

In [18]:
planner_agent = Agent(
    name="Planner Agent",
    instructions=INSTRUCTIONS,
    model=model,
    output_type=WebSearchPlan
)

In [19]:
message = "Latest AI Agent Frameworks in 2025"

with trace("Search"):
    result = await Runner.run(planner_agent, message)
    print(result.final_output)

searches=[WebSearchItem(reason='To find the most current AI agent frameworks available in 2025, including their features and updates.', query='latest AI agent frameworks 2025')]


In [20]:
@function_tool
def send_email(subject: str, html_body: str) -> str:
    """ Send out an email with the given subject and HTML body to all sales prospects """
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email("kaya.atsy@gmail.com")
    to_email = To("kaya.atsy@gmail.com")
    content = Content("text/html", html_body)
    mail = Mail(from_email, to_email, subject, content).get()
    sg.client.mail.send.post(request_body=mail) # type: ignore
    return "Successfully sent email"

In [21]:

INSTRUCTIONS = """You are able to send a nicely formatted HTML email based on a detailed report.
You will be provided with a detailed report. You should use your tool to send one email, providing the 
report converted into clean, well presented HTML with an appropriate subject line."""

email_agent = Agent(
    name="Email Agent",
    instructions=INSTRUCTIONS,
    model=model,
    tools=[send_email]
)

In [22]:
INSTRUCTIONS = (
    "You are a senior researcher tasked with writing a cohesive report for a research query. "
    "You will be provided with the original query, and some initial research done by a research assistant.\n"
    "You should first come up with an outline for the report that describes the structure and "
    "flow of the report. Then, generate the report and return that as your final output.\n"
    "The final output should be in markdown format, and it should be lengthy and detailed. Aim "
    "for 5-10 pages of content, at least 1000 words."
)

In [23]:
class ReportData(BaseModel):
    short_summary: str
    """A short 2-3 sentence summary of the findings."""

    markdown_report: str
    """The final report"""

    follow_up_suggestions: str
    """Suggested topics to research further"""

In [24]:
writer_agent = Agent(
    name="WriterAgent",
    model=model,
    instructions=INSTRUCTIONS,
    output_type=ReportData
)

In [ ]:
async def plan_searches(query:str):
    """Use the planner_agent to plan which searches to run for the query"""
    print("Planning searches...")
    result = await Runner.run(planner_agent, f"Query: {query}")
    print(f"Will perform {len(result.final_output.searches)} searches")
    return result.final_output

async def perform_searches(search_plan: WebSearchPlan):
    """Call search() for each item in the search plan"""
    print("Searching...")
    tasks = [asyncio.create_task(search(item)) for item in search_plan.searches]
    results = await asyncio.gather(*tasks)
    print("Finished Searching")
    return results

async def search(item: WebSearchItem):
    """Use the search agent to run a web search for each item in the search plan"""
    input = f"Search term: {item.query}\nReason for searching: {item.reason}"
    result = await Runner.run(search_agent, input)
    return result.final_output

In [ ]:
async def write_report(query: str, search_results: list[str]):
    """Use the writer agent to write a report based on the search results"""
    print("Thinking about the report...")
    input = f"Original query: {query}\nSummarized search results: {search_results}"
    result = await Runner.run(writer_agent, input)
    print("Finished writing the report.")
    return result.final_output

async def send_email_report(report: ReportData):
    """Use the email agent to send an email with the report"""
    print("Writing email...")
    await Runner.run(email_agent, report.markdown_report)
    print("Email Sent.")
    return report

In [27]:
query = "Latest AI Agent Frameworks in 2025"

with trace("Research trace"):
    print("Starting research...")
    search_plan = await plan_searches(query)
    search_results = await perform_searches(search_plan)
    report = await write_report(query, search_results)
    await send_email_report(report)
    print("Finished the research.")

Starting research...
Planning searches...
Will perform 1 searches
Searching...
Finished Searching
Thinking about the report...
Finished writing the report.
Writing email...
Email Sent.
Finished the research.
